In [128]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.io as sio
#from Lorenz96_RK4 import Lorenz96
from L96_Model import L96
import sys, os, time


In [129]:
# Data, Data Assimilation Hyperparameters
D = 20
Dobs = 12
dims = set(np.arange(20))
#dim_obs = [1, 2, 3, 5, 7, 9, 11, 13, 15, 17, 18, 19]
dim_obs = [0, 1, 2, 4, 6, 8, 10, 12, 14, 16, 17, 18]
dim_unobs = list(dims - set(dim_obs))
M = 200

# Annealing Hyperparameters
Rm = 1
Rf0 = 1e6
alpha = 1.3
betamax = 3


# Hamiltonian Monte Carlo Hyperparameters
niter = int(1e2)
Te = np.exp(1e-1*np.arange(niter))  # Temperature
epsilon = 1e-3*np.ones(niter)
L = 150
mass = [1e3, 1e-2, 1e0]
# Hamiltonian Monte Carlo Tuning Parameters
mass_X = np.zeros(shape=(D,M))
mass_X[dim_obs,:] = mass[0]
mass_X[dim_unobs,:] = mass[1]
mass_nu = mass[2]

mass_X_sqrt = np.zeros(shape=(D,M))
mass_X_sqrt[dim_obs,:] = np.sqrt(2*mass[0])
mass_X_sqrt[dim_unobs,:] = np.sqrt(2*mass[1])
mass_nu_sqrt = np.sqrt(2*mass[2])

# Post-processing
plot_Action_vs_beta = False
savedata = False

In [130]:
# Load the data set
gen_nu = "8.17"
gen_noise = "sig0.4"
gen_dt = "0.001"
gen_delta_t = "0.025"
gen_integrator = "RK4"

# Specify the data path
datapath = ("./L96_D%s_nu%s_%s_dt%s_deltat%s_%s.mat" % 
    (str(D), gen_nu, gen_noise, gen_dt, gen_delta_t, gen_integrator))

if datapath[-3:] == "mat":
    datadict = sio.loadmat(datapath)
    data = datadict["Data"]
    dt = datadict["delta_t"][0]
elif datapath[-3:] == "npy":
    data = np.load(datapath)
else:
    raise IOError ("Does not recognize data file extension\n datapath = %s" % datapath)
    sys.exit()

In [131]:
# Prepare the data
Y = data[dim_obs,:M]

#Initialize the state variables
nu_init = 8

np.random.seed(12345)
X_init = np.zeros((D,M))
X_init[:,0] = 20*np.random.random(size=(D))
X_init[dim_obs, :] = Y

for k in range(0, M-1): # should be M -1??
    X_init[:,k+1] = X_init[:,k] + dt *  L96(X_init[:,k]+dt/2*L96(X_init[:,k], nu_init),nu_init)
    X_init[dim_obs,k+1] = Y[:,k+1]

In [132]:
# Initilize vectorized dirac delta functions
eyeDleft1 = np.roll(np.eye(D), -1, 1)
eyeDleft2 = np.roll(np.eye(D),-2,1)
eyeDright1 = np.roll(np.eye(D),1,1)
eyeD = np.eye(D)
# Some initializations for HMC

# Define the Rf ladder
Rf = Rf0 * (alpha**(np.arange(0,betamax)))

# Initialize the solutions
X_sol = X_init
nu_sol = nu_init
# Initialize the final output cell array. (It is a dict in order to replicate the cells data type in MATLAB)
# This will contain the final X_sol for each beta, and nu_sol for eah beta
q_min = {'X_min': np.zeros(shape=(D,M,betamax)), 'nu_min': np.zeros(shape=(betamax))}

#Initialize action matrix
Action = np.zeros(shape=(betamax, niter))  #in MATLAB code, the shape is (betamax, niter+1). Not sure why the +1
Action_min = np.zeros(shape=(betamax,1))

# Percentage acceptance and percentage downhill
Acceptance = np.zeros(shape=(betamax,1))
Downhill = np.zeros(shape=(betamax, 1))

# Initialize Momentum
pX0 = np.zeros(shape=(D,M))

In [133]:
# Hamiltonian Monte Carlo Algorithm

for beta in range(betamax):
    # Initialize states (i.e. take the results from the previous step)
    X0 = X_sol
    X0[dim_obs,:] = Y  # Is this really necessary?
    nu0 = nu_sol
    
    # Evaluate the starting action under current beta
    Xup1, Xdown2, Xdown1, Xleft1, Zup1, Zdown2, Zdown1, hX = fun_getPieces(X0, nu0, dt)
    Action[beta, 1] = fun_A(X0, Xleft1, hX, Y, dim_obs, M, Rm, Rf[beta])
    Action_min[beta] = Action[beta, 1]
    
    print("Start annealing for beta = %d ..." % beta)
    start_time = time.time()
    
    for n in range(1, niter):
        eps = epsilon[n-1]
        
        # Take current q as starting point
        X = X0
        nu = nu0
        
        # Generate initial momenta from a multivariate normal distribution
        pX0[dim_obs,:] = np.random.normal(0, np.sqrt(mass[0]), size=(Dobs,M))
        pX0[dim_unobs,:] = np.random.normal(0, np.sqrt(mass[1]), size=(D-Dobs, M))
        pnu0 = np.random.normal(0, np.sqrt(mass[2]))
        
        # Get necessary pieces
        Xup1, Xdown2, Xdown1, Xleft1, Zup1, Zdown2, Zdown1, hX = fun_getPieces(X0, nu0, dt)
        # Half step for the X momenta
        pX = pX0 - eps/2*fun_dAX(X, Xup1, Xdown2, Xdown1, Xleft1, Zup1, Zdown2, Zdown1, hX, Y, eyeD, eyeDleft1, eyeDleft2, eyeDright1, D, dim_obs, M, dt, Rm, Rf[beta])
        # Half step for the nu momentum
        pnu = pnu0 - eps/2*fun_dAnu(Xleft1, Zup1, Zdown2, hX, M, dt, Rf[beta])
        
        # Calculate Action when simulation is done
        Action_candidate = fun_A(X, Xleft1, hX, Y, dim_obs, M, Rm, Rf[beta])
        # Metropolis-Hastings acceptance/rejection rule
        if np.random.uniform() < np.exp((Action[beta, n] + np.sum(np.divide(pX0, mass_X_sqrt)**2) # is the index for n right here?
                                        + (pnu0 / mass_nu_sqrt)**2 
                                        - Action_candidate - np.sum(np.divide(pX, mass_X_sqrt)**2)
                                       - (pnu/mass_nu_sqrt)**2 ) / Te[n]): # Check the temperature here
            X0 = X
            nu0 = nu
            Action[beta, n] = Action_candidate
            # Count acceptance rate
            Acceptance[beta] += 1
        else:
            Action[beta, n] = Action[beta, n]
            
        # Check if the current proposal yields the lowest action so far
        if Action[beta, n] < Action_min[beta]:
            Action_min[beta] = Action[beta, n]
            X_sol = X0
            nu_sol = nu0
            Downhill[beta] += 1
            
    print("Done! Time elapsed: %.2fsec" % (time.time() - start_time))
    
    # Record argmin(A[:,beta]) for the current beta
    q_min['X_min'][:,:,beta] = X_sol
    q_min['nu_min'][beta] = nu_sol
    
    # Finalize these acceptances
    Acceptance[beta] /= niter
    Downhill[beta] /= niter
    
print("\n")

Start annealing for beta = 0 ...
Done! Time elapsed: 0.67sec
Start annealing for beta = 1 ...
Done! Time elapsed: 0.67sec
Start annealing for beta = 2 ...
Done! Time elapsed: 0.69sec




In [134]:
test = np.transpose(Xleft1[:,:,None] - hX[:,:,None], (0, 2, 1))
T = np.ones((20,20,200))
test1 = test * T
test2 = np.sum(test1, 0)[None,:,:]
test2.shape


(1L, 20L, 200L)

In [135]:
display(kern1.shape, kern2.shape, kern3.shape)

(20L, 200L)

(20L, 200L)

(20L, 200L, 1L)

In [145]:
# Infer new variables and save useful variables
Action_init = np.zeros(betamax)
for beta in xrange(betamax):
    Xup1, Xdown2, Xdown1, Xleft1, Zup1, Zdown2, Zdown1, hX = fun_getPieces(X_init, nu_init, dt)
    Action_init[beta] = fun_A(X_init, Xleft1, hX, Y, dim_obs, M, Rm, Rf[beta])
    
    
ME = np.zeros(betamax)
FE = np.zeros(betamax)
for beta in xrange(betamax):
    # Evaluate Measurement Error
    ME[beta] = 1/(2*M) * np.sum((q_min['X_min'][dim_obs,:,beta] - Y)**2)
    # Evaluate Model Error
    Xup1, Xdown2, Xdown1, Xleft1, Zup1, Zdown2, Zdown1, hX = fun_getPieces(q_min['X_min'][:,:,beta], q_min['nu_min'][beta], dt)
    kern2 = Xleft1 - hX
    FE[beta] = 1/(2*M) * np.sum((kern2[:,:M-1])) # M-1 intentional here?

    
mass_string = []
for m in mass:
    mass_string.append(m)

In [ ]:
# Save Data
if savedata == 1:
    save()

In [ ]:
def fun_dAX(X, Xup1, Xdown2, Xdown1, Xleft1, Zup1, Zdown2, Zdown1, hX, Y, eyeD, eyeDleft1, eyeDleft2, eyeDright1, D, dim_obs, M, dt, Rm, Rf):
    
    GX = np.multiply(eyeDleft1[:,:,None], np.transpose(Xup1[:,:,None] - Xdown2[:,:,None], (0, 2, 1))) 
    + np.multiply(eyeDright1[:,:,None] - eyeDleft2[:,:,None], np.transpose(Xdown1[:,:,None], (0, 2, 1))) - eyeD[:,:,None]
        
    GZ = np.multiply(eyeDleft1[:,:,None], np.transpose(Zup1[:,:,None] - Zdown2[:,:,None], (0, 2, 1))) 
    + np.multiply(eyeDright1[:,:,None] - eyeDleft2[:,:,None], np.transpose(Zdown1[:,:,None], (0, 2, 1))) - eyeD[:,:,None]
    
    GZGX = np.zeros(shape=(D,D,M))
    for i in xrange(M):
        GZGX[:,:,k] = np.matmul(GZ[:,:,k] , GZ[:,:,k])
    
    T = eyeD[:,:,None] + dt*GZ + dt**2 / 2*GZGX
    kern3 = -Rf/M * np.transpose(np.sum(np.transpose(Xleft1[:,:,None] - hX[:,:,None], (0, 2, 1)) * T,axis=0)[None,:,:], (1, 2, 0))
    kern3[:,M-1] = 0 # What is this for?
    
    kern1 = np.zeros(shape=(D,M))
    kern1[dim_obs,:] = Rm/M * (X[dim_obs,:] - Y)
    
    kern2 = Rf/M * (X - np.roll(hX, 0, 1))
    kern2[:,1] = 0
    
    dAX = kern1 + kern2 + kern3[:,:,0]
    return dAX

In [ ]:
def fun_dAnu(Xleft1, Zup1, Zdown2, hX, M, dt, Rf):
    kern = np.multiply(Xleft1 - hX, dt + dt**2/2*(Zup1 - Zdown2 - 1))
    dAnu = -Rf/M * np.sum(kern)
    
    return dAnu

In [ ]:
def fun_getPieces(X, nu, dt):
    Xup1 = np.roll(X, -1, 0)
    Xdown1 = np.roll(X, 1, 0)
    Xdown2 = np.roll(X, 2, 0)
    Xleft1 = np.roll(X, -1, 1)
    
    Z = X + dt/2* (np.multiply(Xup1 - Xdown2, Xdown1) - X + nu)
    Zup1 = np.roll(Z, -1, 0)
    Zdown1 = np.roll(Z, 1, 0)
    Zdown2 = np.roll(Z, 2, 0)
    
    hX = X + dt * (np.multiply(Zup1 - Zdown2, Zdown1) - Z + nu)
    
    return Xup1, Xdown2, Xdown1, Xleft1, Zup1, Zdown2, Zdown1, hX

In [ ]:
def fun_A(X, Xleft1, hX, Y, dim_obs, M, Rm, Rf):
    kern1 = Rm/(2*M) * np.sum((X[dim_obs,:] - Y)**2)
    
    kern2 = Xleft1 - hX
    kern2 = Rf/(2*M) * np.sum(kern2[:,:M-1]**2) # Is this right? I feel like it's leaving out one column
    
    return kern1 + kern2

In [137]:
def generate_data():
    """
        This function is optional, in case you do not want to use previous data
        
        Args:
            None
        Returns:
            A numpy.array object of shape (D, steps)
    """
    # Set the hyperparamters for generating the data
    # stepsize = dt
    stepsize = 0.01

    # How many time points
    steps = 1000

    # Initial point
    np.random.seed(123456)
    D = 20
    y0 = np.random.random(D)*3
    t0 = 0
    F = 8

    sample = Lorenz96(y0, t0, D, F)
    data = sample.generate(stepsize=stepsize, steps=steps, addNoise=True, noise_level=0.04)
    return data